#### The following KEYWORDS DICTIONARY is extracted from Business Insider artice as mentioned in our Project Scope Document.

https://www.businessinsider.com/things-to-do-in-quarantine-activities-social-distancing-2020-4?amp

In [45]:
Topics = {
"Streaming" : ["Quibi", "HBO", "Netflix", "Free Documentries", "Free Shows"],
"Exercises" : ["Yoga", "Walking", "Running", "Meditate", "Bicycle riding"],
"HBO_Shows" : ["Succession","True Blood","The Sopranos","Veep","The Wire"],
"Virtual_Games" : ["Animal Crossing: New Horizons", "Club Penguin", "Monopoly", "Mario Kart Tour","Words With Friends"],
"Cooking" : ["pancake", "pasta", "baking hacks", "bread", "cake", "cookie"],
"Video_Chat_Platforms" : ["Google hangouts", "Zoom", "QuarantineChat", "HouseParty","Zoom Party"],
"Live_Virtual_Performance" : ["Billie Eilish","BTS", "Dua Lipa", "Sophia Ankel"],
"Cameo" : ["Steven Galanis", "Rachel Dratch", "Mandy Moore", "Restaurant Workers' Relief Fund"],
"Social_Media" : ["Memes", "TikTok", "Instagram"],
"TikTok" : ["airplane","Hibachi grill","Hooters", "Ciara", "Usher", "Jennifer Lopez"],
"Instagram" : ["memes", "video chat", "makeup tutorials", "distance dance", "working_from_home"],
"Tiktok_Meme" : ["highschool musical", "dance workouts", "solo dance", "themed dinners", "distance dance"],
"Virtual_Workouts" : ["Gym", "Yoga", "Dancing", "Toning workout", "HIIT workout"],
}

#### ANACONDA DOESN'T COME PRE-INSTALLED WITH ALL OF THE FOLLOWING PACKAGES. 
Please kindly refer to requirements.txt to install all relevant packages.

In [46]:
#IMPORTING MODULES FOR GOOGLE TRENDS

import os
import pandas as pd
import pprint
from datetime import datetime, timedelta
from pytrends.request import TrendReq
import numpy as np

In [47]:
#INTIALINZING PYTRENDS
pytrends = TrendReq(hl='en-US', tz=360)

#date_range is a parameter used to ping the Google Trends API to get the data for 'last 5 months from today'.
date_range = (datetime.today() - timedelta(150)).strftime('%Y-%m-%d') + " " + datetime.today().strftime('%Y-%m-%d')

**GOOGLE TRENDS API** *takes a list of maximum 5 keywords as input and returns a dataframe for the feature interest_over_time.*


*Some of the Topics in our Keyword Dictionary have more than 5 keywords. Hence, for the purpose of re-usability we've created a function to handle such Topics that have a length exceeding 5.*

In [48]:
#function to iterate over a list in batches of 5 & return a list of "lists with 5 elements"
def keyword_list_creator(keyword_list):
    new_list = []
    for i in range(int(len(keyword_list)/5)+1):
        new_list.append(keyword_list[i*5 : (i+1)*5])
    return(new_list)


In [49]:
#function to return interest_over_time dataframe & related queries from GOOGLE TRENDS 
#for a keyword list of **ONLY 5 elements**

def trend_df(channels):
    
    pytrends.build_payload(channels, cat=0,timeframe = date_range , geo='US', gprop='')

    interest_over_time = pytrends.interest_over_time().drop("isPartial",axis = 1)
    
    try:
        rel_queries = get_google_rel_queries(pytrends.related_queries())
    except:
        print("issue with google_rel_queries")
        rel_queries = []
    return(interest_over_time, rel_queries)


#function to return dataframe from GOOGLE TRENDS for a keyword list of **MORE THAN 5 KEYWORDS**

#uses the trend_df to create and stack  interest_over_time dataframes & Related queries lists 
#so as to return a single output

#Takes a keyword list as an input
#returns a transformed JSON of interest_over_time dataframe and a list of google related queries. 

def create_df(topic_name):
    
    keywords_list = keyword_list_creator(topic_name)
    
    if len(keywords_list[-1]) == 0:
        keywords_list = keywords_list[:-1]

    for i in range(len(keywords_list)):
        if i ==0:
            df, google_rel_queries = trend_df(keywords_list[i])
        else:
            df1, google_rel_queries_1 = trend_df(keywords_list[i])
            df = pd.concat([df, ], axis=1)
            google_rel_queries += google_rel_queries_1


    return(df.to_json(orient='table'),google_rel_queries)


#function to parse the google_related_queries JSON and return a list that contains only the related queries.
def get_google_rel_queries(google_rel_queries):
    rel_queries = []

    for i in google_rel_queries.keys():
#         print(i)
        try :
            rel_queries += list(google_rel_queries[i]['top']["query"][:])
        except:
            continue        

    rel_queries = list(set(rel_queries))
    return(rel_queries)


**GOOGLE TRENDS** *returns a JSON response for related queries. The JSON response is comprised of two parts i.e... 'rising' and 'top'. For the purposes of this project we have limited our scope to only extract the 'top' related queries.*

In [50]:
#IMPORTING RELEVANT MODULES TO INTERACT WITH AZURE COGNITIVE SERVICES(BING SEARCH TRENDS)

import requests
from azure.cognitiveservices.search.websearch import WebSearchClient
from azure.cognitiveservices.search.websearch.models import SafeSearch
from msrest.authentication import CognitiveServicesCredentials
from jsonpath_ng import jsonpath, parse

In [51]:
#FREE SUBSCRIPTION KEY FOR AZURE
subscription_key = "865c1fcf4c99425c9892bddfe34d76ed"
assert subscription_key

In [52]:
#function to connect to bing API and return a list of related queries.

def get_bing_results(keyword_list):

    bing_rel_queries_complete_list = []
    for i in keyword_list:
        headers = {"Ocp-Apim-Subscription-Key": subscription_key}
        params = {"q": i, "textDecorations": True, "textFormat": "HTML"}
        response = requests.get("https://api.cognitive.microsoft.com/bing/v7.0/search", headers=headers, params=params)
        response.raise_for_status()    
        bing_rel_queries_complete_list += rel_query_preprocessing(response.json())
        
    return(bing_rel_queries_complete_list)


#function to parse the bing_related_queries JSON and return a list that contains only the related queries.
#Used JSONPATH to parse the json

def rel_query_preprocessing(bing_search_result_json):
    
    try:
        raw_data = bing_search_result_json['relatedSearches']
        jsonpath_expr = parse('$..text')
        bing_rel_queries = [match.value for match in jsonpath_expr.find(raw_data)]
    
    except KeyError:
        bing_rel_queries = []
        
    bing_rel_queries = list(set(bing_rel_queries))
    
    return(bing_rel_queries)


In [53]:
# get_bing_results(Topics["Instagram"])

### The Output JSON will be created in the following format.


    {"Topic Name" :
         {"interest over time" : #Dataframe

        "related_queries" :
                    {
                      "Google": #list
                      "Bing" :  #list
                      }
         }
    }

In [54]:
#FUNCTION TO CREATE THE FINAL JSON OBJECT IN THE STRUCTURE SHOWN ABOVE.

#Takes the KEYWORDS DICTIONARY as Input.
#Returns a Json Object and a List of "Left over Topics"('Topics that didn't work) as output.
#iterates through the KEYWORDS DICTIONARY.
#makes use of all the above defined functions to create the output JSON object.

def create_final_json(Topics):
    final_json = {}
    worked_topics = []
    for i in Topics.keys():
        
        try:
            final_json[i] = {}
            final_json[i]["related_queries"] = {}
            final_json[i]["interest_over_time"],final_json[i]["related_queries"]["Google"] = create_df(Topics[i])
            try:
                final_json[i]["related_queries"]["Bing"] = get_bing_results(i)
            except:
                print("issue with BING!")
                final_json[i]["related_queries"]["Bing"] = []
                
            worked_topics.append(i)
            print(i,"is processed.")
        except:
            continue
            
    left_out_topics = [x for x in Topics.keys() if x not in worked_topics]
    return(final_json, left_out_topics)

# create_final_json(Topics)

In [55]:
# CREATE THE FINAL JSON
final_json, left_out_topics = create_final_json(Topics)

Streaming is processed.
Exercises is processed.
HBO_Shows is processed.
Virtual_Games is processed.
Cooking is processed.
Video_Chat_Platforms is processed.
Live_Virtual_Performance is processed.
Cameo is processed.
Social_Media is processed.
TikTok is processed.
Instagram is processed.
Tiktok_Meme is processed.
Virtual_Workouts is processed.


In [61]:
# CHECK IF THERE ARE ANY LEFT-OVER TOPICS/KEYWORD LISTS
left_out_topics

[]

In [62]:
# CHECK OUT THE FINAL JSON OBJECT
final_json

{'Streaming': {'related_queries': {'Google': ['watch shows online free',
    'netflix new movies',
    'quibi review',
    'showtime',
    'the outsider',
    'movies on netflix',
    'netflix stock',
    'hbo max',
    'new netflix movies',
    'survive quibi',
    'most dangerous game quibi',
    'amazon prime hbo',
    'watch tv shows free online',
    'netflix movies 2020',
    'free shows online',
    'netflix outer banks',
    'quibi tv',
    'free netflix',
    'new on netflix',
    'dummy quibi',
    'hbo free shows',
    'free shows on amazon prime',
    'reno 911',
    'hbo on hulu',
    'hbo movies',
    'hulu',
    'quibi on tv',
    'quibi ipo',
    'quibi the stranger',
    'watch free tv shows',
    'hbo show',
    'netflix account',
    '123movies',
    'hbo',
    'free movies',
    'quibi price',
    'hbo streaming',
    'the stranger',
    'netflix you',
    'free hbo',
    'broadway shows',
    'quibi meaning',
    'best netflix shows',
    'what is quibi',
    'hbo 

In [59]:
#WRITE OUT THE JSON OBJECT CREATED
import json

with open('DSCI_Project_Output.json', 'w', encoding='utf-8') as f:
    json.dump(final_json, f, ensure_ascii=False, indent=4)

In [64]:
import pipreqs

In [65]:
pipreqs

<module 'pipreqs' from '/Users/pavan/anaconda3/lib/python3.7/site-packages/pipreqs/__init__.py'>